In [22]:
import itertools
import json
import numpy as np
import pandas as pd
import scipy.integrate
import biocircuits
import tqdm

import iqplot
import bokeh.io
bokeh.io.output_notebook()
import panel as pn
pn.extension()

def style(p, autohide=False):
    p.title.text_font="Helvetica"
    p.title.text_font_size="16px"
    p.title.align="center"
    p.xaxis.axis_label_text_font="Helvetica"
    p.yaxis.axis_label_text_font="Helvetica"
    
    p.xaxis.axis_label_text_font_size="13px"
    p.yaxis.axis_label_text_font_size="13px"
    p.background_fill_alpha = 0
    if autohide: p.toolbar.autohide=True
    return p

Loading BokehJS ...

# <center> Homework 9.2: Turing patterns with expanders</center>
<center><img src="werner_turing_circuit.png" width=300px></center>

# a) activator-inhibitor subgraph
\begin{align}
\\[0.1em]
\frac{\partial a}{\partial t} &= D_A\,\frac{\partial^2 a}{\partial x^2}
+ \beta_A\,\frac{(a/k_a)^n}{(a/k_a)^n + (b/k_b)^n} - \gamma_A a \\[1em]
\frac{\partial b}{\partial t} &= D_B\,\frac{\partial^2 b}{\partial x^2}
+ \beta_B\,\frac{(a/k_a)^n}{(a/k_a)^n + (b/k_b)^n} - \gamma_B b.
\end{align}

<center><img src="__9.2_a1.png" width=900px></center>
<center><img src="__9.2_a2.png" width=900px></center>
<center><img src="__9.2_a3.png" width=900px></center>

### 4. Small perturbation from homogeneous steady state
<span style="color: #bbbbbb">*Starting from a small perturbation of the nonzero homogeneous steady state, numerically solve the coupled PDEs. Use no-flux boundary conditions. Plot the resulting steady state concentration profiles for 𝑎 and 𝑏. Do this using the following parameters: 𝑑𝑎=0.033, 𝛽𝑎=2.5, 𝛽𝑏=10, 𝛾𝑎=0.5, and 𝑛=5. Do this five times, once each for the total dimensionless length of the system being 5, 10, 20, 40, and 80. Comment on the pertinence of these results with respect to scaling.*</span>

In [2]:
color_A, color_B, color_E = '#1b9e77', '#d95f02', '#7570b3'

In [3]:
def dc_dt(
    c,
    t,
    x,
    derivs_0,
    derivs_L,
    diff_coeff_fun,
    diff_coeff_params,
    rxn_fun,
    rxn_params,
    n_species,
    h,
):
    """
    Time derivative of concentrations in an R-D system
    for constant flux BCs.

    Parameters
    ----------
    c : ndarray, shape (n_species * n_gridpoints)
        The concentration of the chemical species interleaved in a
        a NumPy array.  The interleaving allows us to take advantage
        of the banded structure of the Jacobian when using the
        Hindmarsh algorithm for integrating in time.
    t : float
        Time.
    derivs_0 : ndarray, shape (n_species)
        derivs_0[i] is the value of the diffusive flux,
        D dc_i/dx, at x = 0, the leftmost boundary of the domain of x.
    derivs_L : ndarray, shape (n_species)
        derivs_0[i] is the value of the diffusive flux,
        D dc_i/dx, at x = L, the rightmost boundary of the domain of x.
    diff_coeff_fun : function
        Function of the form diff_coeff_fun(c_tuple, t, x, *diff_coeff_params).
        Returns an tuple where entry i is a NumPy array containing
        the diffusion coefficient of species i at the grid points.
        c_tuple[i] is a NumPy array containing the concentrations of
        species i at the grid poitns.
    diff_coeff_params : arbitrary
        Tuple of parameters to be passed into diff_coeff_fun.
    rxn_fun : function
        Function of the form rxn_fun(c_tuple, t, *rxn_params).
        Returns an tuple where entry i is a NumPy array containing
        the net rate of production of species i by chemical reaction
        at the grid points.  c_tuple[i] is a NumPy array containing
        the concentrations of species i at the grid poitns.
    rxn_params : arbitrary
        Tuple of parameters to be passed into rxn_fun.
    n_species : int
        Number of chemical species.
    h : float
        Grid spacing (assumed to be constant)

    Returns
    -------
    dc_dt : ndarray, shape (n_species * n_gridpoints)
        The time derivatives of the concentrations of the chemical
        species at the grid points interleaved in a NumPy array.
    """
    # Tuple of concentrations
    c_tuple = tuple([c[i::n_species] for i in range(n_species)])

    # Compute diffusion coefficients
    D_tuple = diff_coeff_fun(c_tuple, t, x, *diff_coeff_params)

    # Compute reaction terms
    rxn_tuple = rxn_fun(c_tuple, t, *rxn_params)

    # Return array
    conc_deriv = np.empty_like(c)

    # Convenient array for storing concentrations
    da_dt = np.empty(len(c_tuple[0]))

    # Useful to have square of grid spacing around
    h2 = h ** 2

    # Compute diffusion terms (central differencing w/ Neumann BCs)
    for i in range(n_species):
        # View of concentrations and diffusion coeff. for convenience
        a = np.copy(c_tuple[i])
        D = np.copy(D_tuple[i])

        # Time derivative at left boundary
        da_dt[0] = D[0] / h2 * 2 * (a[1] - a[0] - h * derivs_0[i])

        # First derivatives of D and a
        dD_dx = (D[2:] - D[:-2]) / (2 * h)
        da_dx = (a[2:] - a[:-2]) / (2 * h)

        # Time derivative for middle grid points
        da_dt[1:-1] = D[1:-1] * np.diff(a, 2) / h2 + dD_dx * da_dx

        # Time derivative at left boundary
        da_dt[-1] = D[-1] / h2 * 2 * (a[-2] - a[-1] + h * derivs_L[i])

        # Store in output array with reaction terms
        conc_deriv[i::n_species] = da_dt + rxn_tuple[i]

    return conc_deriv


def rd_solve(
    c_0_tuple,
    t,
    L=1,
    derivs_0=0,
    derivs_L=0,
    diff_coeff_fun=None,
    diff_coeff_params=(),
    rxn_fun=None,
    rxn_params=(),
    rtol=1.49012e-8,
    atol=1.49012e-8,
):
    """
    Parameters
    ----------
    c_0_tuple : tuple
        c_0_tuple[i] is a NumPy array of length n_gridpoints with the
        initial concentrations of chemical species i at the grid points.
    t : ndarray
        An array of time points for which the solution is desired.
    L : float
        Total length of the x-domain.
    derivs_0 : ndarray, shape (n_species)
        derivs_0[i] is the value of dc_i/dx at x = 0.
    derivs_L : ndarray, shape (n_species)
        derivs_L[i] is the value of dc_i/dx at x = L, the rightmost
        boundary of the domain of x.
    diff_coeff_fun : function
        Function of the form diff_coeff_fun(c_tuple, x, t, *diff_coeff_params).
        Returns an tuple where entry i is a NumPy array containing
        the diffusion coefficient of species i at the grid points.
        c_tuple[i] is a NumPy array containing the concentrations of
        species i at the grid poitns.
    diff_coeff_params : arbitrary
        Tuple of parameters to be passed into diff_coeff_fun.
    rxn_fun : function
        Function of the form rxn_fun(c_tuple, t, *rxn_params).
        Returns an tuple where entry i is a NumPy array containing
        the net rate of production of species i by chemical reaction
        at the grid points.  c_tuple[i] is a NumPy array containing
        the concentrations of species i at the grid poitns.
    rxn_params : arbitrary
        Tuple of parameters to be passed into rxn_fun.
    rtol : float
        Relative tolerance for solver.  Default os odeint's default.
    atol : float
        Absolute tolerance for solver.  Default os odeint's default.

    Returns
    -------
    c_tuple : tuple
        c_tuple[i] is a NumPy array of shape (len(t), n_gridpoints)
        with the initial concentrations of chemical species i at
        the grid points over time.

    Notes
    -----
    .. When intergrating for long times near a steady state, you
       may need to lower the absolute tolerance (atol) because the
       solution does not change much over time and it may be difficult
       for the solver to maintain tight tolerances.
    """
    # Number of grid points
    n_gridpoints = len(c_0_tuple[0])

    # Number of chemical species
    n_species = len(c_0_tuple)

    # Grid spacing
    h = L / (n_gridpoints - 1)

    # Grid points
    x = np.linspace(0, L, n_gridpoints)

    # Set up boundary conditions
    if np.isscalar(derivs_0):
        derivs_0 = np.array(n_species * [derivs_0])
    if np.isscalar(derivs_L):
        derivs_L = np.array(n_species * [derivs_L])

    # Set up parameters to be passed in to dc_dt
    params = (
        x,
        derivs_0,
        derivs_L,
        diff_coeff_fun,
        diff_coeff_params,
        rxn_fun,
        rxn_params,
        n_species,
        h,
    )

    # Set up initial condition
    c0 = np.empty(n_species * n_gridpoints)
    for i in range(n_species):
        c0[i::n_species] = c_0_tuple[i]

    # Solve using odeint, taking advantage of banded structure
    c = scipy.integrate.odeint(
        dc_dt,
        c0,
        t,
        args=params,
        ml=n_species,
        mu=n_species,
        rtol=rtol,
        atol=atol,
    )

    return tuple([c[:, i::n_species] for i in range(n_species)])

In [4]:
def constant_diff_coeffs(c_tuple, t, x, diff_coeffs):
    n = len(c_tuple[0])
    return tuple([diff_coeffs[i] * np.ones(n) for i in range(len(c_tuple))])

def werner_AB_rxn(ab_tuple, t, d_a, β_a, β_b, γ_a, n):
    a, b = ab_tuple
    r_a = β_a * a**n/(a**n + b**n) - γ_a*a
    r_b = β_b * a**n/(a**n + b**n) - b 
    return (r_a, r_b)

def homog_ss_AB(β_a, β_b, γ_a, n):
    a_ss = (β_a / γ_a) * (1 / (1+(γ_a*β_b/β_a)**n))
    b_ss = β_b * (1 / (1+(γ_a*β_b/β_a)**n))
    return a_ss, b_ss

In [5]:
d_a = 0.033
β_a = 2.5
β_b = 10
γ_a = 0.5
n = 5

diff_coeffs = (d_a, 1.0)                   # diffusion coefficients
rxn_params = (d_a, β_a, β_b, γ_a, n)
rxn_fun = werner_AB_rxn

In [10]:
t = np.linspace(0.0, 5_000.0, 1_000)          # time

a_ss, b_ss = homog_ss_AB(β_a, β_b, γ_a, n)

a_0 = np.ones(500) * a_ss                  # steady state
b_0 = np.ones(500) * b_ss                  # steady state

np.random.seed(8176432)                   # seed perturbation
a_0 += 0.0001 * np.random.rand(len(a_0))   # perturbation

In [11]:
L_list = [5, 10, 20, 40, 80]
x_list = [np.linspace(0, L, len(a_0)) for L in L_list]

a_concs_vary_L = np.empty((len(L_list), len(t), len(a_0)))
b_concs_vary_L = np.empty((len(L_list), len(t), len(b_0)))

for i, L in enumerate(L_list):
    concs = rd_solve(
        (a_0, b_0),
        t,
        L=L,
        derivs_0=0,
        derivs_L=0,
        diff_coeff_fun=constant_diff_coeffs,
        diff_coeff_params=(diff_coeffs,),
        rxn_fun=rxn_fun,
        rxn_params=rxn_params
    )
    a_concs = concs[0]
    b_concs = concs[1]
    
    a_concs_vary_L[i] = a_concs
    b_concs_vary_L[i] = b_concs    

In [13]:
t_slider = pn.widgets.FloatSlider(name="t", start=t[0], end=t[-1], value=t[-1], step=t[10]-t[0], width=300)

@pn.depends(t_slider)
def plotter_werner_AB(t_point):
    i = np.searchsorted(t, t_point)
    
    ps = []
    for i_L, x, L in zip(np.arange(len(L_list)), x_list, L_list):
        p = bokeh.plotting.figure(
            title=f"L: {L}",
            width=450, height=300,
            x_axis_label="x/L", y_axis_label="[A], [B]",
            x_range=[0,1],
            y_range=[0, max(a_concs_vary_L.max(), b_concs_vary_L.max())*1.02],
        )
        p.line(x/L_list[i_L], a_concs_vary_L[i_L, i, :], line_width=3, line_color=color_A)
        p.line(x/L_list[i_L], b_concs_vary_L[i_L, i, :], line_width=3, line_color=color_B)
        p = style(p, autohide=True)
        ps.append(p)
    return pn.Column(*ps)

pn.Column(pn.Row(t_slider, align="center"), plotter_werner_AB)

Column
    [0] Row(align='center')
        [0] FloatSlider(end=5000.0, name='t', step=50.05005005005005, value=5000.0, width=300)
    [1] ParamFunction(function)

- doubling length &rarr; approximately doubles number of peaks, does not scale with the length

## sensitivity to initial conditions

In [14]:
def SIC_trajs(
    a_perturb_range, 
    b_perturb_range,
    ss, 
    L_fixed, 
    size_perturb,
    rxn_fun, 
    rxn_params,
    size_a0=500,
):
    a_ss, b_ss = ss
    x_fixed = np.linspace(0, L_fixed, size_a0)
    a_perturb_list = np.linspace(*a_perturb_range, size_perturb)
    b_perturb_list = np.linspace(*b_perturb_range, size_perturb)
    
    a_trajs = np.empty((len(a_perturb_list), len(b_perturb_list), size_a0))
    b_trajs = np.empty((len(a_perturb_list), len(b_perturb_list), size_a0))
    
    for i, a_perturb in enumerate(tqdm.tqdm(a_perturb_list)):
        for j, b_perturb in enumerate(b_perturb_list):
            a_0 = np.ones(size_a0) * a_ss + a_perturb       # steady state
            b_0 = np.ones(size_a0) * b_ss + b_perturb       # steady state
            
            concs = rd_solve(
                (a_0, b_0),
                t,
                L=L_fixed,
                derivs_0=0,
                derivs_L=0,
                diff_coeff_fun=constant_diff_coeffs,
                diff_coeff_params=(diff_coeffs,),
                rxn_fun=rxn_fun,
                rxn_params=rxn_params
            )
            a_concs = concs[0]
            b_concs = concs[1]

            a_trajs[i, j] = a_concs[-1,:]
            b_trajs[i, j] = b_concs[-1,:]
    return x_fixed, a_perturb_list, b_perturb_list, a_trajs, b_trajs

def plots_grid_trajs(x_fixed, a_perturb_list, b_perturb_list, a_trajs, b_trajs):
    ps = []
    for i, a_perturb in enumerate(a_perturb_list):
        for j, b_perturb in enumerate(b_perturb_list):
            p = bokeh.plotting.figure(
                height=100, width=100, 
                y_range=(-0.05, max(a_trajs.max(), b_trajs.max())*1.2)
            )
            p.line(x_fixed, a_trajs[i, j], line_color=color_A, line_width=3)
            p.line(x_fixed, b_trajs[i, j], line_color=color_B, line_width=3)
            p.toolbar_location=None
            p.xaxis.visible, p.yaxis.visible=False, False

            ps.append(p)
    return ps


def count_sign_changes(arr):
    count = 0
    sign_val = lambda x: '+' if x >= 0 else '-'
    
    sign_old = sign_val(arr[0])
    for a in arr:
        sign_new = sign_val(a)
        if sign_new == sign_old: pass
        else: count += 1
        sign_old = sign_new
        
    return count

def wave_classifier(traj):
    """
    Classifies single trajectory
    0: single centered peak
    1: 1.5 pulses, half ends on left
    2: 1.5 pulses, half ends on right
    3: double centered peak
    """
    traj_diff = np.diff(traj)
    sign_changes = count_sign_changes(traj_diff)
    if sign_changes == 1:
        return 0
    elif sign_changes == 2:
        if traj[0] > traj[-1]: return 1
        if traj[0] < traj[-1]: return 2
    elif sign_changes == 3:
        return 3
    else: 
        return -1


def retrieve_waveforms(a_perturb_list, b_perturb_list, a_trajs):
    """
    Returns waveforms grid of integers where
    0: single centered peak
    1: 1.5 pulses, half ends on left
    2: 1.5 pulses, half ends on right
    3: double centered peak
    -1: flat
    """
    
    waveforms = np.empty((len(a_perturb_list), len(b_perturb_list)))
    for i, a_perturb in enumerate(a_perturb_list):
        for j, b_perturb in enumerate(b_perturb_list):
            waveforms[i, j] = wave_classifier(a_trajs[i, j])
            
    return waveforms

def plot_classified_waveforms(
    a_perturb_list, b_perturb_list, waveforms, 
    color_single, color_double, 
    color_L_one_half, color_R_one_half,
):
    """
    Returns heatmap of different waveforms
    """
    color_dict = {
        0: color_single, 
        1: color_L_one_half, 
        2: color_R_one_half, 
        3: color_double, 
        -1: "black"
    }
    
    # rectangle dimensions
    width = b_perturb_list[1]-b_perturb_list[0]
    height = a_perturb_list[1]-a_perturb_list[0]
    
    p = bokeh.plotting.figure(
        height=550, width=800, title="Perturbation Sensitivity: Final Waveforms",
        x_axis_label="B perturbation", y_axis_label="A perturbation"
    )
    _xs, _ys, _colors = [], [], []
    for i, a_perturb in enumerate(a_perturb_list):
        for j, b_perturb in enumerate(b_perturb_list):
            _xs.append(b_perturb)
            _ys.append(a_perturb)
            _colors.append(color_dict[waveforms[i, j]])
    _df = pd.DataFrame({'x':_xs,'y':_ys,'color':_colors})
    p.rect(
        source=_df,
        x='x', y='y', 
        width=width, height=height,
        color='color'
    )
    legend = bokeh.models.Legend(items=[
        ("single centered peak", [p.rect(width=width, height=height, color=color_single)]), 
        ("1.5 peaks, ends left", [p.rect(width=width, height=height, color=color_L_one_half)]), 
        ("1.5 peaks, ends right", [p.rect(width=width, height=height, color=color_R_one_half)]), 
        ("double centered peaks", [p.rect(width=width, height=height, color=color_double)])
    ], location="center")
    p.add_layout(legend, "right")
            
    return style(p)

In [15]:
a_ss, b_ss = homog_ss_AB(β_a, β_b, γ_a, n)
ss = (a_ss, b_ss)

a_perturb_range=(1e-5, 1e-2)
b_perturb_range=(1e-5, 1e-2)

size_perturb = 10

L_fixed = 5

rxn_params = (d_a, β_a, β_b, γ_a, n)
rxn_fun = werner_AB_rxn

*This takes a little less than 11 seconds to run.*

In [16]:
x_fixed, a_perturb_list, b_perturb_list, a_trajs, b_trajs = SIC_trajs(
    a_perturb_range, b_perturb_range, 
    ss, L_fixed, size_perturb, rxn_fun, rxn_params
)

100%|██████████| 10/10 [00:28<00:00,  2.86s/it]


In [17]:
ps = plots_grid_trajs(x_fixed, a_perturb_list, b_perturb_list, a_trajs, b_trajs)
bokeh.io.show(bokeh.layouts.gridplot(ps, ncols=size_perturb))

- brief glimpse into sensitivity of waveforms to initial conditions -- very sensitive!
- waveforms bound by boundary conditions (must have zero slope / flux at x = 0 and x = L

This is one of the poorer visualizations I've made (lack of axes, but read it as x-axis is the range of perturbations of the inhibitor B and the y-axis as the range of perturbations of the activator A.)

In [18]:
color_single = "#c05a48"
color_double = "#669a8b"
color_L_one_half = "#A2A928"
color_R_one_half = "#D2B153"
color_waveforms1 = (color_single, color_double, color_L_one_half, color_R_one_half)

color_single = "#808bbe"
color_double = "#325676"
color_L_one_half = "#d4b3cf"
color_R_one_half = "#b09dc9"
color_waveforms2 = (color_single, color_double, color_L_one_half, color_R_one_half)

color_single = "#991a40"
color_double = "#1c0517"
color_L_one_half = "#D296A0"
color_R_one_half = "#9db0bb"
color_waveforms3 = (color_single, color_double, color_L_one_half, color_R_one_half)

waveforms = retrieve_waveforms(a_perturb_list, b_perturb_list, a_trajs)
p1 = plot_classified_waveforms(a_perturb_list, b_perturb_list, waveforms, *color_waveforms1)
p2 = plot_classified_waveforms(a_perturb_list, b_perturb_list, waveforms, *color_waveforms2)
p3 = plot_classified_waveforms(a_perturb_list, b_perturb_list, waveforms, *color_waveforms3)

palette_selector_10 = pn.widgets.Select(options=['pudding','quinone', 'perl'], value='quinone', width=200)
@pn.depends(palette_selector_10)
def _plotter_10(palette):
    if palette == "pudding": return p1
    elif palette == "perl": return p2
    elif palette == "quinone": return p3
pn.Column(pn.Row(pn.Spacer(width=200), palette_selector_10), _plotter_10)

Column
    [0] Row
        [0] Spacer(width=200)
        [1] Select(options=['pudding', 'quinone', ...], value='quinone', width=200)
    [1] ParamFunction(function)

Hmmm. Not sure if this is a bug (in my code) or a feature (of the system)... but it looks totally random!

*Note that this demonstration will not match the commentary in the paper, since my L = 5, which results in four distinct waveforms, whereas they observe six.*

Let's increase the resolution. The code to create this plot is below; it takes around half an hour to run. The plot is shown below

In [19]:
a_perturb_range=(1e-5, 1e-2)
b_perturb_range=(1e-5, 1e-2)
size_perturb = 100

a_perturb_list = np.linspace(*a_perturb_range, size_perturb)
b_perturb_list = np.linspace(*b_perturb_range, size_perturb)

In [20]:
# x_fixed, a_perturb_list, b_perturb_list, a_trajs, b_trajs = SIC_trajs(
#     a_perturb_range, b_perturb_range, 
#     ss, L_fixed, size_perturb, rxn_fun, rxn_params
# )

# waveforms = retrieve_waveforms(a_perturb_list, b_perturb_list, a_trajs)

# ## .... SAVING AS JSON FILE ....
# _lst = [float(_) for _ in np.ravel(waveforms)]
# d_waveforms = {'integers': _lst}
# with open("waveforms.json", 'w') as f:
#     json.dump(d_waveforms, f)

# p = plot_classified_waveforms(a_perturb_list, b_perturb_list, waveforms, *color_waveforms)
# bokeh.io.show(p)

In [23]:
## .... LOADING IN JSON FILE ....
with open('waveforms.json', 'r') as f:
    d_waveforms = json.load(f)
waveforms100 = np.array(d_waveforms['integers']).reshape((100, 100))

p1 = plot_classified_waveforms(a_perturb_list, b_perturb_list, waveforms100, *color_waveforms1)
p2 = plot_classified_waveforms(a_perturb_list, b_perturb_list, waveforms100, *color_waveforms2)
p3 = plot_classified_waveforms(a_perturb_list, b_perturb_list, waveforms100, *color_waveforms3)

In [24]:
palette_selector_100 = pn.widgets.Select(options=['pudding','quinone', 'perl'], value='quinone', width=200)
@pn.depends(palette_selector_100)
def _plotter_100(palette):
    if palette == "pudding": return p1
    elif palette == "perl": return p2
    elif palette == "quinone": return p3
pn.Column(pn.Row(pn.Spacer(width=200), palette_selector_100), _plotter_100)

Column
    [0] Row
        [0] Spacer(width=200)
        [1] Select(options=['pudding', 'quinone', ...], value='quinone', width=200)
    [1] ParamFunction(function)

In [26]:
_items, _counts = np.unique(np.ravel(waveforms100), return_counts=True)
_classifiers = ['single centered peak', '1.5 peaks, ends left',
                '1.5 peaks, ends right', 'double centered peak']

for _classifier, _count in zip(_classifiers, _counts[1:]):
    print(f'# occurences {_classifier} : \t', _count)

# occurences single centered peak : 	 4758
# occurences 1.5 peaks, ends left : 	 1522
# occurences 1.5 peaks, ends right : 	 1797
# occurences double centered peak : 	 1922


- predominantly single-centered peak (fits one waveform), then 1.5, then 2

In [235]:
# L_list = [5, 10, 20, 40, 80]
# X_list = [np.linspace(0, L, len(a_0)) for L in L_list]

# L = L_list[0]                              # physical length of system
# x = X_list[0]                          

# concs = rd_solve(
#     (a_0, b_0),
#     t,
#     L=L,
#     derivs_0=0,
#     derivs_L=0,
#     diff_coeff_fun=constant_diff_coeffs,
#     diff_coeff_params=(diff_coeffs,),
#     rxn_fun=rxn_fun,
#     rxn_params=rxn_params
# )
# a_concs = concs[0]
# b_concs = concs[1]

In [236]:
# t_slider = pn.widgets.FloatSlider(name="t", start=t[0], end=t[-1], value=t[-1], step=t[1]-t[0], width=300)

# @pn.depends(t_slider)
# def plotter_werner_AB(t_point):
#     i = np.searchsorted(t, t_point)
    
#     ps = []
#     for i_L, x, L in zip(np.arange(len(L_list)), x_list, L_list):
#         p = bokeh.plotting.figure(
#             title=f"L: {L}",
#             width=450, height=300,
#             x_axis_label="x/L", y_axis_label="[A], [B]",
#             x_range=[0,1],
#             y_range=[0, max(a_concs_vary_L.max(), b_concs_vary_L.max())*1.02],
#         )
#         p.line(x/L_list[i_L], a_concs_vary_L[i_L, i, :], line_width=3, line_color=color_A)
#         p.line(x/L_list[i_L], b_concs_vary_L[i_L, i, :], line_width=3, line_color=color_B)
#         p = style(p, autohide=True)
#         ps.append(p)
#     return pn.Column(*ps)

# pn.Column(pn.Row(t_slider, align="center"), plotter_werner_AB)

# b) activator-inhibitor-expander

<center><img src="__9.2_b1.jpg" width=1200px></center>

2. 

<span style="color: #bbbbbb">*There are no homogeneous steady states for this system. For our initial “steady state” when doing the numerical calculations, we will take 𝑒0=1 and solve for the values of 𝑎 and 𝑏 such that d𝑎/d𝑡=d𝑏/d𝑡=0. Nevermind that for this “steady state,” d𝑒/d𝑡≠0. Again, starting from a small perturbation of this “steady state,” numerically solve the coupled PDEs using no-flux boundary conditions. Plot the resulting steady state concentration profiles for 𝑎 and 𝑏. Do this using the same parameters as in part (a-4), with additional parameters 𝑑𝑒=0.33, 𝜅𝑎=0.5, and 𝜅𝑒=1. Again, do this five times, once each for the total dimensionless length of the system being 5, 10, 20, 40, and 80. (You can do it for more lengths if you like.) Comment on the pertinence of these results with respect to scaling.*</span>

\begin{align}
a_{\mathrm{ss}} &= \cfrac{\beta_a}{\kappa_a}\cfrac{1}{1+(\kappa_a \beta_b/ \beta_a)^n} \\[0.5em]
b_{\mathrm{ss}} &= \beta_b \cfrac{1}{1+(\kappa_a \beta_b / \beta_a)^n} \\[0.5em]
e_{\mathrm{ss}} &= 1
\end{align}

In [27]:
def werner_rxn(abe_tuple, t, d_a, d_e, β_a, β_b, n, κ_a, κ_e):
    a, b, e = abe_tuple
    r_a = β_a * a**n/(a**n + b**n) - κ_a*e*a
    r_b = β_b * a**n/(a**n + b**n) - e*b
    r_e = 1 - κ_e*e*b
    return (r_a, r_b, r_e)

def homog_ss_ABE(β_a, β_b, κ_a, n):
    κββn = (κ_a * β_b / β_a)**n
    a_ss = β_a / κ_a / (1+κββn)
    b_ss = β_b / (1+κββn)
    e_ss = 1
    return a_ss, b_ss, e_ss

In [28]:
d_a = 0.033
d_e = 0.33
β_a = 2.5
β_b = 10
n = 5
κ_a = 0.5
κ_e = 1

diff_coeffs = (d_a, 1.0, d_e)                   # diffusion coefficients
rxn_params = (d_a, d_e, β_a, β_b, n, κ_a, κ_e)  
rxn_fun = werner_rxn

In [40]:
t = np.linspace(0.0, 10_000.0, 10_000)          # time
t = np.concatenate((np.array([0]), np.logspace(-1, 5, 500)))          # time

a_ss, b_ss, e_ss = homog_ss_ABE(β_a, β_b, κ_a, n)

a_0 = np.ones(500) * a_ss                  # steady state
b_0 = np.ones(500) * b_ss                  # steady state
e_0 = np.ones(500) * e_ss                  # steady state

a_0 += 0.00012                             # perturbation

In [41]:
L_list = [5, 10, 20, 40, 80]
x_list = [np.linspace(0, L, len(a_0)) for L in L_list]

a_concs_vary_L = np.empty((len(L_list), len(t), len(a_0)))
b_concs_vary_L = np.empty((len(L_list), len(t), len(b_0)))
e_concs_vary_L = np.empty((len(L_list), len(t), len(e_0)))

for i, L in enumerate(L_list):
    concs = rd_solve(
        (a_0, b_0, e_0),
        t,
        L=L,
        derivs_0=0,
        derivs_L=0,
        diff_coeff_fun=constant_diff_coeffs,
        diff_coeff_params=(diff_coeffs,),
        rxn_fun=rxn_fun,
        rxn_params=rxn_params
    )
    a_concs = concs[0]
    b_concs = concs[1]
    e_concs = concs[2]
    
    a_concs_vary_L[i] = a_concs
    b_concs_vary_L[i] = b_concs 
    e_concs_vary_L[i] = e_concs   

In [42]:
i = np.searchsorted(t, t[-1])

ps = []
for i_L, x, L in zip(np.arange(len(L_list)), x_list, L_list):
    p = bokeh.plotting.figure(
        title=f"L: {L}",
        width=450, height=300,
        x_axis_label="x/L", y_axis_label="[A], [B]",
        x_range=[0,1],
#         y_range=[-0.05, max(a_concs_vary_L.max(), b_concs_vary_L.max())*1.02],
        
    )
    p.line(x/L_list[i_L], a_concs_vary_L[i_L, i, :], line_width=3, line_color=color_A)
    p.line(x/L_list[i_L], b_concs_vary_L[i_L, i, :], line_width=3, line_color=color_B)
    p.line(x/L_list[i_L], e_concs_vary_L[i_L, i, :], line_width=3, line_color=color_E)
    
    p = style(p, autohide=True)
    ps.append(p)
bokeh.io.show(bokeh.layouts.layout(*ps))

- turn into widget/movie
- normalize and put all on same plot (A B E separate though?)
- examine more length scales!

In [ ]:
# t_slider = pn.widgets.FloatSlider(name="t", start=t[0], end=t[-1], value=t[-1], step=t[1]-t[0], width=300)

# @pn.depends(t_slider)
# def plotter_werner_AB(t_point):
#     i = np.searchsorted(t, t_point)
    
#     ps = []
#     for i_L, x, L in zip(np.arange(len(L_list)), x_list, L_list):
#         p = bokeh.plotting.figure(
#             title=f"L: {L}",
#             width=450, height=300,
#             x_axis_label="x/L", y_axis_label="[A], [B]",
#             x_range=[0,1],
#             y_range=[0, max(a_concs_vary_L.max(), b_concs_vary_L.max())*1.02],
#         )
#         p.line(x/L_list[i_L], a_concs_vary_L[i_L, i, :], line_width=3, line_color=color_A)
#         p.line(x/L_list[i_L], b_concs_vary_L[i_L, i, :], line_width=3, line_color=color_B)
#         p = style(p, autohide=True)
#         ps.append(p)
#     return pn.Column(*ps)

# pn.Column(pn.Row(t_slider, align="center"), plotter_werner_AB)

- 16x  the length -> only 2x the frequency... cool!

# c)
<span style="color: #bbbbbb">*Comment qualitatively on how the expander works to scale the Turing patterns. By what other means (other than mutual inhibition of B and inhibition of A) might an expander operate? Remember, these models are postulates of what might be happening in developing organisms, so it is useful to dream up alternatives.*</span>

how does it work?
- κ_a is 
alternatives?
- try the circuit where A is activating E?

# ideas
- varying levels of E with system size
- How does the changes in volume when stretching the length change the concentrations? is it valid to treat gamma as a constant? should the degradation term have a factor of $L$ in it? 
        Rewrite a model where
        cell division without cell growth 
        cell division with cell growth
        is there any sensible way to write a decay term that's dependent on L?
        try varying diffusion with L

In [ ]:
pattern
turn
saturn

morphogenesis 
bliss
amiss
abyss
resists

expander
meander
salamander
commander

Meet the salamander commander: Mr. Expander